In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error, mean_absolute_error

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:

data_in = pd.read_csv('data/test.csv', delimiter=';')
data_int = pd.read_csv('data/train.csv', delimiter=';')
data = pd.concat([data_in, data_int])

data['position'].nunique()

data['pos1'] = data['position'].apply(lambda x: x.split(',')[0].lower())

data['pos1'].nunique()

data['pos2'] = data['position'].apply(lambda x: x.split(',')[1].lower() if len(x.split(','))>1 else 'None')

data['pos3'] = data['position'].apply(lambda x: x.split(',')[2].lower() if len(x.split(','))>2 else 'None')

data = data[['id','pos1','pos2','pos3','salary_desired']].copy()

lst = ['специалист', 'not_top50', 'продавец', 'слесарь', 'юрист', 'повар',
       'механик', 'медсестра', 'рабочий', 'бухгалтер', 'менеджер',
       'водитель', 'строитель', 'дизайнер', 'управляющий', 'охранник',
       'инженер', 'мастер', 'аналитик', 'монтажник', 'кассир', 'машинист',
       'электромонтер', 'экономист', 'секретарь', 'администратор',
       'воспитатель', 'кладовщик', 'начальник', 'оператор пк',
       'соцработник', 'нефтяник', 'врач', 'ведущий специалист', 'любая',
       'программист', 'уборщица', 'госслужащий', 'психолог', 'швея',
       'офис-менеджер', 'кадровик', 'торговый представитель', 'техник',
       'менеджер по работе с клиентами', 'электрогазосварщик',
       'диспетчер', 'оператор', 'парикмахер', 'бригадир',
       'помощник руководителя']

def lstadd(x):
    for l in lst:
        if l in x:
            return l
    return 'not_top50'

res2 = data.groupby('pos1')['salary_desired'].mean().to_dict()
res4 = data.groupby('pos2')['salary_desired'].mean().to_dict()
res6 = data.groupby('pos3')['salary_desired'].mean().to_dict()

# data['pos1_salary'] = data['pos1'].apply(lambda x: res1[x])
data['pos1_salary_desired'] = data['pos1'].apply(lambda x: res2[x])
# data['pos2_salary'] = data['pos2'].apply(lambda x: res3[x])
data['pos2_salary_desired'] = data['pos2'].apply(lambda x: res4[x])
# data['pos3_salary'] = data['pos3'].apply(lambda x: res5[x])
data['pos3_salary_desired'] = data['pos3'].apply(lambda x: res6[x])

data['pos1'] = data['pos1'].apply(lambda x: lstadd(x))
data['pos2'] = data['pos2'].apply(lambda x: lstadd(x))
data['pos3'] = data['pos3'].apply(lambda x: lstadd(x))

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
ohe.fit(data['pos1'].values.reshape(-1, 1))
pos1_ohe = pd.DataFrame(ohe.transform(data['pos1'].values.reshape(-1, 1)).toarray())
pos1_ohe.columns = ohe.get_feature_names(['pos1'])

ohe = OneHotEncoder()
ohe.fit(data['pos2'].values.reshape(-1, 1))
pos2_ohe = pd.DataFrame(ohe.transform(data['pos2'].values.reshape(-1, 1)).toarray())
pos2_ohe.columns = ohe.get_feature_names(['pos2'])

ohe = OneHotEncoder()
ohe.fit(data['pos3'].values.reshape(-1, 1))
pos3_ohe = pd.DataFrame(ohe.transform(data['pos3'].values.reshape(-1, 1)).toarray())
pos3_ohe.columns = ohe.get_feature_names(['pos3'])

pos_ohe = pd.merge(pos1_ohe,pos2_ohe,left_index=True,right_index=True)
pos_ohe = pos_ohe.merge(pos3_ohe,left_index=True,right_index=True)

data.drop(['pos1','pos2','pos3','salary_desired'],axis=1,inplace=True)
data = data.merge(pos_ohe,left_index=True,right_index=True)

data.to_csv('mean_salary_by_poss_test.csv',index=False)

C:\Users\SafiullinaEA\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
